In [24]:
import csv
import urllib
import os
import re
import sys 
import requests
reload(sys)
sys.setdefaultencoding('UTF-8')

commSite = 'http://www.dailymail.co.uk/reader-comments/p/asset/readcomments/'
start = '?max=100&offset='
end ='&order=desc&rcCache=shout'
comPattern = r"\{\"id\":(.*?),\"dateCreated\":\"(.*?)T(.*?)\..*?\",\"message\":\"(.*?)\",\"assetId\":(\d*?),\"assetUrl\":\"(.*?)\",\"assetHeadline\":\"(.*?)\",\"assetCommentCount\":(.*?),\"userAlias\":\"(.*?)\",\"userLocation\":\"(.*?)\",\"userIdentifier\":\"(\d*?)\",\"voteRating\":(\d*?),\"voteCount\":(\d*?),\"hasProfilePicture\":(.*?),\"replies\":{\"totalCount\":(.*?),\"comments"
headers = 'id', 'datecreated', 'timeCreated', 'message', 'assetId', 'assetURL', 'assetHeadline', 'assetCommentCount', 'userAlias', 'userLocation', 'userIdentifier', 'voteRating', 'voteCount', 'hasProfilePicture', 'comments', 'parent'
headersReq = {
    'Host': 'www.dailymail.co.uk',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Cookie': 'OX_plg=swf|shk|pm; dm_clientsegment=a; ISAPPersistent=%7B%22__upcv%22%3A%2276742326366754755435%22%2C%22__spcv%22%3A%22cb%3D2017%26s127108%3D1_23%22%7D; __gads=ID=9519a55ec611a565:T=1485171751:S=ALNI_MY9tDNLHn6wwlI8FS4nZA0ketRC_g; __qca=P0-1783442771-1485264680494; latest-article-count=2509748; GED_PLAYLIST_ACTIVITY=W3sidSI6IllNTUEiLCJ0c2wiOjE0ODU1Mzk4MzksIm52IjowLCJ1cHQiOjE0ODU0MzAxNjIsImx0IjoxNDg1NDMwMzIxfV0.; __SW=342d49e243b55da3c190974f1f493969; s_pers=%20s_nr_v15%3D1485171803546%7C1487763803546%3B%20s_c39_v15%3D%252Fhome%252Fhome%7C1485432057720%3B%20s_fid%3D0712944FE1CE9FE1-059CD9E4E85112AD%7C1548618059405%3B%20s_visit_v15%3D1%7C1485547859433%3B%20s_pn2_v15%3D%252Fhome%252Fhome%7C1485547859463%3B%20gpv_c1%3D%252Fhome%7C1485547859470%3B%20gpv_c2%3D%252Fhome%7C1485547859485%3B%20gpv_c4%3Dhome%7C1485547859502%3B%20s_vmonthnum%3D1485900000096%2526vn%253D42%7C1485900000096%3B%20s_monthinvisit%3Dtrue%7C1485547859529%3B%20s_vweeknum%3D1485640800155%2526vn%253D42%7C1485640800155%3B%20s_mweekinvisit%3Dtrue%7C1485547859553%3B%20s_vdaynum%3D1485554400006%2526vn%253D3%7C1485554400006%3B%20s_dayinvisit%3Dtrue%7C1485547859617%3B; s_sess=%20s_pn_v15%3D%252Fhome%252Fhome%3B%20s_cc%3Dtrue%3B%20s_sq%3D%3B; cto_dailymail=; __utma=141568423.1023705450.1485171745.1485542912.1485546067.41; __utmc=141568423; __utmz=141568423.1485268706.6.3.utmcsr=localhost:8888|utmccn=(referral)|utmcmd=referral|utmcct=/notebooks/DM.ipynb; asi_segs=D11729|D12217|D0; rsi_segs=D05509_11729|D05509_12217|D05509_0; baseline-article-count=2512405; autorefresh=channel; cs_autorefresh=channel'
}

startReg = r"^(\{\"status\":\"success\",\"code\":\"\d+\",\"payload\":\{\"total\":\d+,\"parentCommentsCount\":\d+,\"offset\":\"\d+\",\"max\":\d+,\"page\":\[)"
endReg = r"(\]\,\"assetId\":\"\d+\",\"assetStatusId\":.+,\"isOldArticle\":.+\,\"shoutDisabledChannels\":\"\{ .+ \[\] \}\"\}\})$"

URL = raw_input('Enter Full Daily Mail Article URL: ')
article = re.search('\/article-(\d+)\/',URL).group(1)

outfile  = open(article+'.csv', "wb")
writer = csv.writer(outfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
writer.writerow(headers)

def findComment (matching):
        match = re.search (comPattern,matching,re.I)
        if match: writer.writerow(match.groups())
        return

reqURL = commSite + article
req = requests.get (reqURL, headers = headersReq)
html_source = req.text

numOfComments = re.findall ('{"total":(\d+)',html_source)
nums = int(numOfComments[0])

#Get comments
for i in range (0, nums,100): 
    req = requests.get (reqURL + start + str(i) + end, headers = headersReq)
    html_source = req.text
    html_source = re.sub(startReg,'',html_source)
    html_source = re.sub(endReg,'',html_source)
    html_source.strip()
    matchings = re.findall('(\{\"id\":.*?comments)',html_source)
    for matching in matchings:
        findComment (matching)
        
outfile.close()
      
req.connection.close()

Enter Full Daily Mail Article URL: http://www.dailymail.co.uk/news/article-4173688/148m-lotto-winner-says-lover-left-press.html
